In [24]:
from bertopic import BERTopic
import pandas as pd
import pickle
from sklearn.datasets import fetch_20newsgroups#
import datetime as dt

In [25]:
submissions1 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions1.json")
submissions2 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions2.json")
submissions3 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions3.json")
submissions4 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions4.json")
submissions5 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions5.json")
submissions6 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions6.json")
submissions7 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions7.json")

KeyboardInterrupt: 

In [ ]:
submissions = pd.concat([submissions1, submissions2, submissions3, submissions4, submissions5, submissions6, submissions7])

In [ ]:
#create a column with real datetimes
submissions['created_utc'] = submissions['created_utc'].apply(lambda date: dt.datetime.fromtimestamp(date))
submissions = submissions.sort_values(by='created_utc', ascending=True)

In [28]:
posts = submissions[submissions['created_utc'].dt.year==2014].sample(200)
# curr_df = {"january": posts[posts['created_utc'].dt.month==1],
#            "febuary": posts[posts['created_utc'].dt.month==2],
#            "march": posts[posts['created_utc'].dt.month==3],
#             "april": posts[posts['created_utc'].dt.month==4],
#             "may": posts[posts['created_utc'].dt.month==5],
#             "june": posts[posts['created_utc'].dt.month==6],
#             "august": posts[posts['created_utc'].dt.month==8],
#             "september": posts[posts['created_utc'].dt.month==9],
#             "october": posts[posts['created_utc'].dt.month==10],
#             "november": posts[posts['created_utc'].dt.month==11],
#             "december": posts[posts['created_utc'].dt.month==12]}


In [29]:
docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data = pd.Series(docs['data']).sample(5000).to_list() # huely cut down the data because it takes forever otherwise. This is slow but w/e
data[:5]

['Is it going to be possible to upgrade a C610 to Tempest?\nIf so, how...motherboard switch?  \nProbably gonna be expensive right?',
 '\nThis is getting sad.  All you can do is make this ridiculous statements,\nbased upon some old information and a Press Democrat article that was\npoorly written.\n\nPlease show the numbers for your use of "MUCH more".  I have not seen\nthem.  And I want them to be true and accurate, or at least show a \ntrend within the everyday gay population.\n\nThere are all kinds of \'damaged and screwed up\' people, and most of\nthem are not gay.',
 'I am looking for a 8 meg 72-pin SIMM for my Centris 610.  Where is the\nbest place to purchase one (stock, shipping, warrenty), and if\npossible, phone numbers so that I can order one as soon as possible.\n\nThanks,\n\nSteven\n\n-- \n"And hope that there\'s intelligent life somewhere out in space"  Monty Python\n" \'Cause there\'s bugger all down here on Earth"           The Meaning of Life',
 "\n\n\tThe defenition of

In [12]:
# precalculate sentence embeddings so they don't have to be done each time and save them
from sentence_transformers import SentenceTransformer
if 0:
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # fast but pretty accurate embeddings
    embeddings = embedding_model.encode(data, show_progress_bar=True)
    
    with open("embeddings.pkl", "wb") as fOut:
        pickle.dump({'embeddings': embeddings},fOut)

Batches: 100%|██████████| 157/157 [08:03<00:00,  3.08s/it]


In [13]:
# Load embeddings from file
embeddings = None
if 1:
    with open("embeddings.pkl", "rb") as fIn:
        cache_data = pickle.load(fIn)
        embeddings = cache_data['embeddings']

In [36]:
# Default representations. Want to try another representation that has less stopwords
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2") 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-11-10 17:35:53,528 - BERTopic - Reduced dimensionality
2023-11-10 17:35:53,783 - BERTopic - Clustered reduced embeddings


In [33]:
# A maybe better representation?
from bertopic.representation import KeyBERTInspired
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2", representation_model=KeyBERTInspired()) 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-11-10 17:33:50,128 - BERTopic - Reduced dimensionality
2023-11-10 17:33:50,752 - BERTopic - Clustered reduced embeddings


In [34]:
topicModel.save("keyBertModel", serialization='safetensors')

In [35]:
topicModel.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1580,-1_to_have_from_for,"[to, have, from, for, by, about, of, more, bee...","[\nbackupable, if restored to the same machine..."
1,0,458,0_openwindows_what_about_are,"[openwindows, what, about, are, more, to, thos...",[Frequently-asked questions about the OPEN LOO...
2,1,368,1_privacy_think_information_have,"[privacy, think, information, have, don, inter...",[I dreamed that the great judgment morning had...
3,2,236,2_jews_information_what_who,"[jews, information, what, who, think, about, p...","[Is life a pass/fail course, and does God grad..."
4,3,182,3_comics_wolverine_spider_more,"[comics, wolverine, spider, more, why, this, 1...","[I'm not very impressed by the old so-called ""..."
...,...,...,...,...,...
57,56,12,56_conference_tampa_pctools_chicago,"[conference, tampa, pctools, chicago, blues, d...",[This is message is only of interest to those ...
58,57,12,57_computational_belief_data_learning,"[computational, belief, data, learning, biolog...","[ri\n\n\n\nI do agree with you, in a way. The..."
59,58,12,58_fundamentalists_beliefs_belief_prevent,"[fundamentalists, beliefs, belief, prevent, fr...","[\nI had a '82 Saab 900 Turbo, and now have a ..."
60,59,11,59_blinking_leafs_radios_mhz,"[blinking, leafs, radios, mhz, transmit, flyer...",[Where can I buy blinking LEDs besides radio s...


In [18]:
topicModel.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'nhl | hockey | league | baseball | pittsburgh', 462],
                                   [1, 'motorcycle | riding | bikes | driving | bike', 415],
                                   [2, 'scripture | bible | christians | faith | christian', 301],
                                   [3, 'nutrition | health | vitamin | medicine | patients', 240],
                                   [4, 'encryption | nsa | crypto | encrypted | privacy', 165],
                                   [5, 'nasa | spacecraft | space | astro | shuttle', 163],
                                   [6, 'hi |  |  |  | ', 156],
                                   [7, 'palestinians | israelis | israeli | palestinian | zionist', 138],
                                   [8, 'dos | os | windows | ms | sys', 102],
                                   [9, 'firearms | guns | militia | armed | gun', 100],
                                   [10, 'libxmu | x11r5 | xmu | xterm | x11', 90],
                                   [11, 'what | reading | read | funny | he', 84],
                                   [12, 'jpeg | formats | quicktime | graphics | images', 61],
                                   [13, 'tanks | fires | tank | fire | survivors', 57],
                                   [14, 'vga | vram | card | cards | vlb', 55],
                                   [15, 'armenians | armenian | armenia | azerbaijanis | azerbaijani', 55],
                                   [16, 'vocoder | modems | telephone | encryption | phones', 49],
                                   [17, 'processor | pentium | cpu | mhz | performance', 48],
                                   [18, 'mail | mailing | addresses | email | list', 48],
                                   [19, 'iraqi | iraq | kuwait | bombing | bosnia', 40],
                                   [20, 'amplifiers | amp | amplifier | amps | stereo', 37],
                                   [21, 'seagate | harddisk | drives | disk | hd', 37],
                                   [22, 'modem | uart | serial | 16550 | ethernet', 37],
                                   [23, 'sam | hear | speak | spell | listen', 33],
                                   [24,
                                    'xcreatewindow | xcreatesimplewindow | xsetstandardproperties | defaultrootwindow | window',
                                    32],
                                   [25, 'morality | morals | moral | immoral | ethics', 31],
                                   [26, 'grounded | grounding | conductors | wiring | conductor', 31],
                                   [27,
                                    'heterosexual | homosexuality | homosexual | sexuality | homosexuals',
                                    30],
                                   [28, 'monitors | monitor | vga | resolution | viewsonic', 30],
                                   [29, 'enforcement | batf | warrant | waco | assault', 30],
                                   [30, 'plots | plotting | 3d | polygon | sphere', 29],
                                   [31, 'modem | modems | tcm3105 | fax | baycom', 28],
                                   [32, 'hardware | prices | sale | price | disk', 27],
                                   [33, 'scsi | disk | 5mb | hd | drives', 26],
                                   [34, 'stephanopoulos | congress | president | secretary | senator', 26],
                                   [35, 'printer | printers | printing | print | printouts', 25],
                                   [36, 'seminar | conference | presentations | siggraph | courses', 24],
                                   [37,
                                    'homosexuality | homosexual | homosexuals | leviticus | heterosexual',
                                    24],
                                   [38, 'radars | radar | surveillance | detectors | detector', 20],
                                   [39, 'monitor | monitors | vga | lcd | screen', 19],
